# Pairwise MLP approach

Get tracksters from a certain neighbourhood.

Train a NN to decide whether two tracksters should be joined.

Neighbourhood:
- get links from ticlNtuplizer/graph
    - figure out how these links are formed
- convert the tracksters into some latent space and predict a link between them
- later extend this using edgeconv or sageconf to add information from the neighbourhood

Graph:
- linked_inners
    - nodes linked to the given tracksters within its cone


## MLP

In [1]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split, DataLoader


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

from reco.dataset import TracksterPairs

In [2]:
# Apple silicon setup
# this ensures that the current MacOS version is at least 12.3+
# print(torch.backends.mps.is_available())
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [3]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
ds = TracksterPairs("data", N_FILES=10, balanced=True)
ds

Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2834.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_542.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_112.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2137.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_10.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2567.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2588.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2072.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_954.root
Processing: /Users/ecuba/data/multiparticle_complet/new_ntuples_14992862_2422.root


<TracksterPairs len=20938 balanced=True max_distance=10 energy_threshold=10>

In [5]:
ds.x = torch.nn.functional.normalize(ds.x, p=torch.inf, dim=0)
ds.x

tensor([[0.1667, 0.0833],
        [0.0833, 0.1667],
        [0.2500, 0.7500],
        ...,
        [0.0833, 0.0833],
        [0.0833, 0.0833],
        [0.0833, 0.0833]])

In [6]:
print("dataset balance:", float(sum(ds.y) / len(ds.y))) 

dataset balance: 0.5


In [7]:
loss_obj = torch.nn.BCEWithLogitsLoss()

def train(model, opt, loader):
    epoch_loss = 0
    for batch, labels in loader:
        # reset optimizer and enable training mode
        opt.zero_grad()
        model.train()

        # move data to the device
        batch = batch.to(device)
        labels = labels.to(device)
        
        # get the prediction tensor
        z = model(batch).reshape(-1)

        # compute the loss
        loss = loss_obj(z, labels)
        epoch_loss += loss

        # back-propagate and update the weight
        loss.backward()
        opt.step()

    return float(epoch_loss)

@torch.no_grad()
def test(model, data):
    total = 0
    correct = 0
    for batch, labels in data:
        model.eval()
        batch = batch.to(device)
        labels = labels.to(device)
        z = model(batch).reshape(-1)
        prediction = (z > 0.5).type(torch.int)
        total += len(prediction) 
        correct += sum(prediction == labels.type(torch.int))
    return (correct / total)

In [8]:
ds_size = len(ds)
test_set_size = ds_size // 10
train_set_size = ds_size - test_set_size
train_set, test_set = random_split(ds, [train_set_size, test_set_size])
print(f"Train samples: {len(train_set)}, Test samples: {len(test_set)}")

train_dl = DataLoader(train_set, batch_size=64, shuffle=True)
test_dl = DataLoader(test_set, batch_size=64, shuffle=True)

Train samples: 18845, Test samples: 2093


In [9]:
model = nn.Sequential(
    nn.Linear(ds.x.shape[1], 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Dropout()
)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

for epoch in range(0, 101):
    loss = train(model, optimizer, train_dl)
    scheduler.step()
    if epoch % 20 == 0:
        train_acc = test(model, train_dl)
        test_acc = test(model, test_dl)
        print(f'Epoch: {epoch}, loss: {loss:.4f}, train acc: {train_acc:.4f}, test acc: {test_acc:.4f}')

Epoch: 0, loss: 186.5649, train acc: 0.6482, test acc: 0.6441
Epoch: 20, loss: 178.8527, train acc: 0.6993, test acc: 0.6980
Epoch: 40, loss: 178.7077, train acc: 0.6993, test acc: 0.6980
Epoch: 60, loss: 177.9001, train acc: 0.6993, test acc: 0.6980
Epoch: 80, loss: 179.0440, train acc: 0.6993, test acc: 0.6980
Epoch: 100, loss: 178.6754, train acc: 0.6993, test acc: 0.6980


In [10]:
pred = []
lab = []
for b, l in test_dl:
    pred += (model(b) > 0.5).type(torch.int).tolist()
    lab += l.tolist()

tn, fp, fn, tp = confusion_matrix(lab, pred).ravel()
print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")
print(f'Accuracy: {accuracy_score(lab, pred):.4f}')
print(f'Precision: {precision_score(lab, pred):.4f}')
print(f'Recall: {recall_score(lab, pred):.4f}')

TP: 521, TN: 940, FP: 101, FN: 531
Accuracy: 0.6980
Precision: 0.8376
Recall: 0.4952
